In [1]:
import pandas as pd
import os
import pymysql

csv_files = os.listdir('dataset')
df_list = [pd.read_csv(os.path.join('dataset', file)) for file in csv_files]

combined_df = pd.concat(df_list, ignore_index=True)

combined_df.to_csv(r'C:\Users\Administrator\Desktop\code\assignment\assignmentbus_route_full.csv', index=False)

df = pd.read_csv("bus_routes.csv")

df.insert(0, 'id', range(1, len(df) + 1))

df['Price'] = df['Price'].str.replace('INR ', '')

df['Seat_Availability'] = df['Seat_Availability'].str.extract(r'(\d+)')
print(df['Seat_Availability'])

connection = pymysql.connect(host='localhost', user='root', password='root', database="redbus")
cursor = connection.cursor()

df = df.dropna()


column_definitions = ", ".join(
    f"`{col}` {dtype}" for col, dtype in zip(df.columns, df.dtypes.astype(str))
).replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT")

table_name = "bus_routes"

cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
result = cursor.fetchone()

if result is None:
    create_table_query = f"CREATE TABLE {table_name} ({column_definitions})"
    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")
else:
    print(f"Table '{table_name}' already exists.")

for _, row in df.iterrows():
    try:
        insert_query = f"INSERT INTO {table_name} VALUES {tuple(row)}"
        cursor.execute(insert_query)
    except Exception as e:
        print(f"Error inserting row: {row}\nError: {e}")

connection.commit()

connection.close()


0         2
1         1
2         2
3         4
4         3
         ..
19325    54
19326    24
19327    61
19328    48
19329    48
Name: Seat_Availability, Length: 19330, dtype: object
Table 'bus_routes' already exists.
